# Stop Data
This file will read in the csv data from a file, 
clean and prepare it 
enter it in the stops database

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

# Read data from csv file 

In [9]:
stop_file = "static/data/Police_Stop_data.csv"
stop_df=pd.read_csv(stop_file)
stop_df.head()

,OBJECTID,masterIncidentNumber,responseDate,reason,problem,callDisposition,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,x,y,policePrecinct,neighborhood,lastUpdateDate
0,1001,16-406479,2016-11-09T17:43:21.000Z,NaN,Attempt Pick-Up (P),BKG-Booking,NaN,YES,NO,Native American,Native American,Male,44.949552,-93.281346,-1.038403e+07,5.613583e+06,5.0,Whittier,2017-08-08T10:24:35.000Z
1,1002,16-406495,2016-11-09T17:58:28.000Z,NaN,Suspicious Vehicle (P),ADV-Advised,NaN,NO,NO,Unknown,Black,Male,45.014157,-93.308244,-1.038703e+07,5.623750e+06,4.0,Cleveland,2017-08-08T10:24:35.000Z
2,1003,16-406507,2016-11-09T18:04:48.000Z,NaN,Suspicious Vehicle (P),GOA-Gone on Arrival,NaN,NaN,NaN,NaN,NaN,NaN,44.995210,-93.299420,-1.038604e+07,5.620767e+06,4.0,Near - North,2017-08-08T10:25:31.000Z
3,1004,16-406526,2016-11-09T18:20:12.000Z,NaN,Traffic Law Enforcement (P),NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.982470,-93.244000,-1.037987e+07,5.618762e+06,2.0,Marcy Holmes,2017-08-08T10:25:31.000Z
4,1005,16-406555,2016-11-09T18:40:01.000Z,NaN,Suspicious Vehicle (P),AOK- All OK,NaN,NO,NO,Unknown,Unknown,Unknown,44.911719,-93.216454,-1.037681e+07,5.607634e+06,3.0,Minnehaha,2017-08-08T10:25:03.000Z


# Imports for database


In [3]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Set up tables

In [4]:
# Define our neighborhoodData table
class neighborhoodData(Base):
    __tablename__ = 'neighborhoodData'
    id = Column(Integer, primary_key=True)
    neighborhood = Column(String)

In [5]:
# Define our stopData table
class stopData(Base):
    __tablename__ = 'stopData'
    OBJECTID = Column(Integer, primary_key=True)
    neighborhood = Column(String)
    responseDate = Column(String)
    citationIssued = Column(String)
    lat= Column(Integer)
    long= Column(Integer)
    gender =Column(String)
    responseDow = Column(Integer)
    responseDay =Column(Integer)
    responseMonth = Column(Integer)
    responseMonthName =Column(String)
    responseYear = Column(Integer)
    

In [6]:
# Define dowData table
class dowData(Base):
    __tablename__ = 'dowData'
    id = Column(Integer, primary_key=True)
    neighborhood = Column(String)
    gender =Column(String)
    responseDow = Column(Integer)
    genderCount = Column(Integer)

In [7]:
Base.metadata.tables
# Create our database engine
engine = create_engine('sqlite:///stops.sqlite')
# This is where we create our tables in the database
Base.metadata.create_all(engine)
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

# Reduce Columns
Reduce the columns to those needed and eliminate rows with null gender  
check count to confirm later operations  

In [10]:
      #select needed columns
select_df=stop_df.loc[:,["OBJECTID", "responseDate","citationIssued","lat","long","gender","neighborhood"]]

# remove null gender rows
reducedA_df=select_df.dropna(axis=0,subset=['gender'])

# remove null neighborhood rows
reduced_df=reducedA_df.dropna(axis=0,subset=['neighborhood'])
reduced_df

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NaN,44.949552,-93.281346,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NaN,45.014157,-93.308244,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NaN,44.911719,-93.216454,Unknown,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NaN,44.976087,-93.277436,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NaN,44.959130,-93.288030,Male,Lowry Hill East
7,1008,2016-11-09T18:54:58.000Z,NaN,44.962763,-93.233781,Unknown,Seward
8,1009,2016-11-09T19:11:59.000Z,NaN,45.020463,-93.308902,Male,Cleveland
9,1010,2016-11-09T19:28:56.000Z,NaN,44.984679,-93.307326,Male,Near - North
10,1011,2016-11-09T19:41:16.000Z,NaN,44.986110,-93.249890,Male,Marcy Holmes
11,1012,2016-11-09T20:01:14.000Z,NaN,45.003680,-93.283380,Female,Hawthorne


# Get a list of neighborhoods
remove 1 null neighborhood

In [11]:
#Get a listing and count of neighborhoods
neighbor_df=reduced_df["neighborhood"]
neighborhood_df=pd.DataFrame(neighbor_df.drop_duplicates())

neighborhood_df.tail()

,neighborhood
1270,Cooper
1317,Bryn - Mawr
2595,University of Minnesota
4425,Field
5897,Humboldt Industrial Area


In [ ]:
neighborhood_df.to_csv("static/data/neighborhood_data.csv", index=False,header=True)

# Put neighborhood data in the table

In [ ]:
neighborhood_df.to_sql(neighborhoodData.__tablename__, engine, if_exists='replace')

session.commit()

# modify Gender 
combine gender Unknown and Gender Non-Conforming into gender Other

In [13]:
pd.options.mode.chained_assignment = None  # default='warn'
#fill in Unknown vaue with Other
reduced_df.loc[reduced_df.gender=="Unknown","gender"] = "Other"
      # fill in Gender Non-Conforming values with other
reduced_df.loc[reduced_df.gender =="Gender Non-Conforming","gender"] = "Other"
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NaN,44.949552,-93.281346,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NaN,45.014157,-93.308244,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NaN,44.911719,-93.216454,Other,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NaN,44.976087,-93.277436,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NaN,44.959130,-93.288030,Male,Lowry Hill East


In [14]:
#what gender values are present ?
#verify count
gender=reduced_df.groupby(["gender"]).count()
gender

,OBJECTID,responseDate,citationIssued,lat,long,neighborhood
gender,,,,,,
Female,19717,19717,12523,19717,19717,19717
Male,55334,55334,36284,55334,55334,55334
Other,13398,13398,8912,13398,13398,13398


# Modify citationIssued 
change NaN to NO

In [15]:
# change null citationIssued to NO
reduced_df[["citationIssued"]]=reduced_df[["citationIssued"]].fillna(value='NO')
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NO,44.949552,-93.281346,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NO,45.014157,-93.308244,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NO,44.911719,-93.216454,Other,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NO,44.976087,-93.277436,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NO,44.959130,-93.288030,Male,Lowry Hill East


In [16]:
#what gender values are present ?
# verify count
citation=reduced_df.groupby(["citationIssued"]).count()
citation

,OBJECTID,responseDate,lat,long,gender,neighborhood
citationIssued,,,,,,
NO,79183,79183,79183,79183,79183,79183
YES,9266,9266,9266,9266,9266,9266


# Parse date
responseDow = day of the week  
responseDay = day of the month as zero padded decimal  
responseDonth = month as a zero padded decimal  
responseMonthName= month as abbreviated name  
responseYear  

In [17]:
#parse date
date_list= [datetime.strptime( entry,"%Y-%m-%dT%H:%M:%S.000Z") for entry in reduced_df["responseDate"]]
# determine response day of week
dow_list= [datetime.strftime(entry,"%w") for entry in date_list]

# split off response day, month and year
day_list = [datetime.strftime(entry,"%d") for entry in date_list]
month_list=[datetime.strftime(entry,"%m") for entry in date_list]
month_name_list=[datetime.strftime(entry,"%b") for entry in date_list]
year_list= [datetime.strftime(entry,"%Y") for entry in date_list]

#add column to reduced_df
reduced_df.loc[:,"responseDow"]=dow_list
reduced_df.loc[:,"responseDay"]=day_list
reduced_df.loc[:,"responseMonth"]=month_list
reduced_df.loc[:,"responseMonthName"]=month_name_list
reduced_df.loc[:,"responseYear"]=year_list
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
0,1001,2016-11-09T17:43:21.000Z,NO,44.949552,-93.281346,Male,Whittier,3,09,11,Nov,2016
1,1002,2016-11-09T17:58:28.000Z,NO,45.014157,-93.308244,Male,Cleveland,3,09,11,Nov,2016
4,1005,2016-11-09T18:40:01.000Z,NO,44.911719,-93.216454,Other,Minnehaha,3,09,11,Nov,2016
5,1006,2016-11-09T18:40:44.000Z,NO,44.976087,-93.277436,Male,Downtown West,3,09,11,Nov,2016
6,1007,2016-11-09T18:51:47.000Z,NO,44.959130,-93.288030,Male,Lowry Hill East,3,09,11,Nov,2016
7,1008,2016-11-09T18:54:58.000Z,NO,44.962763,-93.233781,Other,Seward,3,09,11,Nov,2016
8,1009,2016-11-09T19:11:59.000Z,NO,45.020463,-93.308902,Male,Cleveland,3,09,11,Nov,2016
9,1010,2016-11-09T19:28:56.000Z,NO,44.984679,-93.307326,Male,Near - North,3,09,11,Nov,2016
10,1011,2016-11-09T19:41:16.000Z,NO,44.986110,-93.249890,Male,Marcy Holmes,3,09,11,Nov,2016
11,1012,2016-11-09T20:01:14.000Z,NO,45.003680,-93.283380,Female,Hawthorne,3,09,11,Nov,2016


# Deliver clean stop data
to file "static/data/clean_data.csv"  
or to database

In [18]:
reduced_df.to_csv("static/data/clean_stop_data.csv", index=False,header=True)

# Add Stop data to the database

In [ ]:
reduced_df.to_sql(stopData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

In [19]:
#test for data
neighbor = session.query(stopData.neighborhood, stopData.gender).filter_by(neighborhood="Como").first()
neighbor

('Como', 'Male')

# Prep responseDow data
get count by neighborhood, responseDow and gender

In [20]:
# get count of genders by neighborhood and responseDow
group_df=reduced_df.groupby(["neighborhood","responseDow","gender"]).count()
group_df.head()

OBJECTID  responseDate  citationIssued  lat  \
neighborhood responseDow gender                                                
Armatage     0           Female         1             1               1    1   
                         Male           9             9               9    9   
                         Other          9             9               9    9   
             1           Female         2             2               2    2   
                         Male           6             6               6    6   

                                 long  responseDay  responseMonth  \
neighborhood responseDow gender                                     
Armatage     0           Female     1            1              1   
                         Male       9            9              9   
                         Other      9            9              9   
             1           Female     2            2              2   
                         Male       6            6              6   

                                 responseMonthName  responseYear  
neighborhood responseDow gender                                   
Armatage     0           Female                  1             1  
                         Male                    9             9  
                         Other                   9             9  
             1           Female                  2             2  
                         Male                    6             6

In [21]:
# remove indexing rename columns and reduce columns
new_group=group_df.reset_index()
next_group=new_group.rename(columns={"OBJECTID":"genderCount"})
dow_group=next_group[["neighborhood","responseDow","gender","genderCount"]]
dow_group.head()

,neighborhood,responseDow,gender,genderCount
0,Armatage,0,Female,1
1,Armatage,0,Male,9
2,Armatage,0,Other,9
3,Armatage,1,Female,2
4,Armatage,1,Male,6


# Deliver clean day of week data  
to file "static/data/dow_clean_data.csv" 
or database

In [22]:
# write ad csv file -- json file doesn't come out right
dow_group.to_csv("static/data/clean_dow_data.csv" ,index=False, header=True)

# Add dow Data to database

In [ ]:
dow_group.to_sql(dowData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()